# Carbon prices and forest preservation over space and time in the Brazilian Amazon


In [1]:
import os, sys
import math
import pickle
!pip install casadi
import casadi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
!git clone https://github.com/Finalblue777/amazon_mhpy
workdir = os.getcwd() + '/amazon_mhpy/'
sys.path.insert(0, workdir+'mcmc')
from mcmc_sampling import create_hmc_sampler

# Local Debugging flag; remove when all tested
_DEBUG = True

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


Cloning into 'amazon_mhpy'...


remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.


remote: Compressing objects: 100% (54/54), done.


remote: Total 73 (delta 21), reused 70 (delta 18), pack-reused 0
Receiving objects: 100% (73/73), 3.13 MiB | 23.39 MiB/s, done.
Resolving deltas: 100% (21/21), done.


# 1 Parameter Uncertainty

# 1.1 Max-min problem
We investigate a static formulation of robustness to parameter uncertainty.  For each site, we consider the parameter pair $\beta^i = (\gamma^i, \theta^i)$ for $i=1,2,...,I$, where $θ_i$ is a site-specific productivity parameter, $γ^i \geq 0$ denotes
the density of CO2e that is present in a primary forest in site $i$. Let
$\mathbf{\beta}$ denote full parameter vector including all sites and hence of dimension $2 \times I.$

Our planner is uncertain about these parameter values and instead has baseline
probability distribution $\pi.$  In addition, this planner is uncertain about what  distribution to use and instead  thinks of $\pi$ as a rough approximation.  We address this uncertainty by
introducing ambiguity about the parameter distribution.

Let $d$ be the vector of decisions and $f( \mathbf{d},\mathbf{\beta})$ for the resulting value  given the unknown parameter $\mathbf{\beta}.$  We use a divergence measure to capture  ambiguity about the parameter distribution.  For $\int g( \mathbf{\beta}) d\pi( \mathbf{\beta}) = 1,$ the relative entropy (or Kullback-Leibler) divergence

$$
\int_{\mathcal B} \log g(\mathbf{\beta}) g( \mathbf{\beta}) d \pi(\mathbf{ \beta}) \ge 0,
$$

is a commonly used measure of divergence between a probability $g(\beta)  d\pi(  \beta)$ and the baseline $d\pi(\beta).$
 To produce optimal controls that are robust to the parameter uncertainty, solve

$$
\max_{ d} \min_{g, \int g d\pi = 1} \int_{\mathcal B} f(\mathbf{ d},  \mathbf{ \beta} )   g(\mathbf{ \beta}) d\pi( \mathbf{\beta})
+ \xi \int \log g( \mathbf{\beta}) g(\mathbf{ \beta} ) d\pi( \mathbf{\beta})
$$

where $\xi > 0$ is penalty parameter. Alternatively, we can think of $\xi$ a as Lagrange multiplier on a relative entropy divergence constraint.

We implement a full commitment to the baseline distribution by  making $\xi$  arbitrarily large.  More modest settings capture a concern for robustness.

(WE SHOULD WRITE THE CODE SOLVE SEPERATELY THE MAX AND THE MIN PROBLEM)

## 1.2 Worst case probability

One nice feature of using relative entropy divergence to explore distributional sensitivity is that the minimization problem has a quasi-analytical solution:

$$
-\xi \log \int_{\mathcal B} \exp\left[ - {\frac 1 \xi } f(\mathbf{ d},  \mathbf{ \beta} )  \right]  d\pi( \beta) = \min_{g, \int g d\pi = 1} \int_{\mathcal B} f(\mathbf{ d},  \mathbf{ \beta} )   g(\mathbf{ \beta}) d\pi(\mathbf {\beta})
+ \xi \int \log g(\mathbf{ \beta}) g( \mathbf{\beta}) d\pi( \mathbf{\beta})\tag{1}
$$

where the minimizing $g$ given by:

$$
g^* = \frac {{\exp\left[ - {\frac 1 \xi } f(\mathbf{d},  \mathbf{\beta} )\right]}}
{{\int_{\mathcal B} \exp\left[ - {\frac 1 \xi } f(\mathbf{d}, \tilde  {\mathbf{\beta}})\right]d\pi( \tilde {\mathbf{\beta}} )}}\tag{2}
$$

This tilts the distribution towards smaller objectives for each given decision $\mathbf{d}$. The candidate solution presumes that:

$$
\int_{\mathcal B} \exp\left[ - {\frac 1 \xi } f(\mathbf{d},  \mathbf{\beta})\right]d\pi(\mathbf{\beta} ) < \infty
$$

which implicitly limits how fat the tail can be for the distribution $\pi$.

For conceptual reasons, we also switch the order of the maximization and minimization.  Under quite general conditions, we can invoke the min-max theorem:

$$
\min_{g, \int g d\pi = 1} \max_{\mathbf{d}}  \int_{\mathcal B} f(\mathbf{d},  \mathbf{\beta} )   g(\mathbf{\beta}) d\pi(\mathbf{\beta})
+ \xi \int \log g(\mathbf{\beta}) g(\mathbf{\beta}) d\pi(\mathbf{\beta})
$$
where $\xi > 0$ is penalty parameter.

Consider the inner maximization problem:

$$
\max_{\mathbf{d}}  \int_{\mathcal B} f(\mathbf{d},  \mathbf{\beta} )   g(\mathbf{\beta}) d\pi(\mathbf{\beta})
$$

where we are free to drop the relative entropy penalty as it does not depend on the decision $\mathbf{d}$.  Provided that this problem has a solution for the outer $g$ minimization, then the planner is maximizing against this particular (penalized) "worst case probability."

This computation is of interest as a way to interpret the consequences of any given choice of the penalty parameter $\xi$.  In practice, we find it revealing to explore alternative choices of $\xi$ and deduce their implications for the implied worst case probabilities. This follows a common practice for robust Bayesian methods.

## 1.3 Numerical Solution
For solving the robust problem numerically, we take an iterative approach, also supported by the min-max theorem.

Specifically, we proceed as follows:

1. Given a $g$, we solve the maximization problem for a candidate $\mathbf{d}.$  We ignore the relative entropy penalty term in this solution.

$$
\max_{ d}\int_{\mathcal B} f(\mathbf{ d},  \mathbf{ \beta} )   g(\mathbf{ \beta}) d\pi( \mathbf{\beta})
+ \xi \int \log g( \mathbf{\beta}) g(\mathbf{ \beta} ) d\pi( \mathbf{\beta}),\text{ given }g
$$

2. For a given $\mathbf{d}$, we solve the minimization problem with the relative entropy term to obtain a new candidate for $g$.

$$
 \min_{g, \int g d\pi = 1} \int_{\mathcal B} f(\mathbf{ d},  \mathbf{ \beta} )   g(\mathbf{ \beta}) d\pi( \mathbf{\beta})
+ \xi \int \log g( \mathbf{\beta}) g(\mathbf{ \beta} ) d\pi( \mathbf{\beta}), \text{ given }\mathbf{ d}
$$

where

$$
g^* = \frac {{\exp\left[ - {\frac 1 \xi } f(\mathbf{d},  \mathbf{\beta} )\right]}}
{{\int_{\mathcal B} \exp\left[ - {\frac 1 \xi } f(\mathbf{d}, \tilde  {\mathbf{\beta}})\right]d\pi( \tilde {\mathbf{\beta}} )}}
$$
3. We repeat the steps until we achieve convergence.

For step 2, we use a Hamiltonian simulation approach along with the quasi-analytical solution in computing equation [2].  A numerical method is necessary because of the denominator term, and Markov simulation gives us one way to explore a parameter space of potentially large dimension.  Very similar to how a Metropolis-Hastings algorithm can be used  to compute a Bayesian posterior in terms of a prior and a likelihood, we calculate the exponentially tilted solution using $d\pi(\mathbf{\beta})$ and $\exp\left[-{\frac 1 \xi} f(\mathbf{d}, \mathbf{\beta})\right].$


# 2 Optimization Preparation

## 2.1 Load Site Data Function

The function `load_site_data(site_num, norm_fac=1.0)` is designed to load and preprocess site-specific data, including the prior for $\beta^i = (\gamma^i, \theta^i)$. The function takes two parameters:

- `site_num` : An integer representing the site number.
- `norm_fac` : A float representing the normalization factor. Its default value is 1.0.

The function starts by reading a CSV file specific to the site number passed as the `site_num` parameter. The file should be named as "calibration_{n}SitesModel.csv", where `{n}` is the site number.

The function then extracts information from several columns in the data frame. The columns are named using the site number and the corresponding information type (e.g., `z_2017_{n}Sites`, `zbar_2017_{n}Sites`, etc.).

The extracted information includes:

- `z_2017` : Data specific to the site from 2017.
- `zbar_2017` : Mean data specific to the site from 2017.
- `gamma` : Gamma value for the site.
- `gammaSD` : Standard deviation of the gamma value for the site.
- `forestArea_2017_ha` : Forest area for the site in 2017 (in hectares).
- `theta` : Theta value for the site.

After extracting the data, the function normalizes `zbar_2017` and `z_2017` using the normalization factor `norm_fac`.

Finally, the function returns a tuple containing the normalized `zbar_2017`, `gamma`, `gammaSD`, `z_2017`, `forestArea_2017_ha`, and `theta` data.


In [2]:
site_num = 10

def load_site_data(site_num,
                   norm_fac=1.0,
                  ):
    """
    Load site data

    :returns:
        -
    """
    # Read data file
    n=site_num
    file=workdir+f"/data/calibration_{n}SitesModel.csv"
    df = pd.read_csv(file)

    # Extract information
    z_2017             = df[f'z_2017_{n}Sites'].to_numpy()
    zbar_2017          = df[f'zbar_2017_{n}Sites'].to_numpy()
    gamma              = df[f'gamma_{n}Sites'].to_numpy()
    gammaSD            = df[f'gammaSD_{n}Sites'].to_numpy()
    forestArea_2017_ha = df[f'forestArea_2017_ha_{n}Sites'].to_numpy()
    theta              = df[f'theta_{n}Sites'].to_numpy()

    # Normalize Z data
    zbar_2017 /= norm_fac
    z_2017    /= norm_fac

    return (zbar_2017, gamma, gammaSD, z_2017, forestArea_2017_ha, theta)

zbar_2017, gamma, gammaSD, z_2017, forestArea_2017_ha, theta = load_site_data(site_num,norm_fac=1e+9)
print("gamma", gamma)
print("gammaSD", gammaSD)
print("z_2017", z_2017)
print("gamma - gammaSD", gamma - gammaSD)
print("zbar_2017", zbar_2017)
print("theta", theta)

gamma [528.37325404 775.47438931 395.26648222 374.48793533 264.38850068
 629.55636429 525.40049002 647.83704716 655.24936145 515.34274728]
gammaSD [238.11314476 326.38539421 186.89564803 181.8379563  134.00071021
 271.3797981  249.13116229 258.88173247 291.55140499 205.48669328]
z_2017 [6.39629863e-03 6.67577070e-04 1.60344839e-02 9.49710184e-03
 6.04574833e-03 2.70778687e-03 7.79948366e-03 5.73571943e-05
 6.77236678e-03 1.30112628e-03]
gamma - gammaSD [290.26010928 449.0889951  208.37083419 192.64997903 130.38779047
 358.17656618 276.26932772 388.95531469 363.69795645 309.856054  ]
zbar_2017 [0.04586502 0.03280152 0.03713347 0.02052573 0.01063091 0.07698995
 0.02063136 0.03375215 0.05417736 0.06184899]
theta [1.55266026 1.8118974  1.92270587 3.22535906 1.91376031 0.99658051
 2.59539362 0.05511117 1.99328104 0.28637799]


## 2.2 Log Density Function of Posterior Density

The function `log_density_function()` evaluates the log-density of the objective or posterior distribution. This function is used within an optimization loop, with some parameters updated in each iteration of the loop.
To be specific, the function calculate the log of
$$ \exp\left[ - {\frac 1 \xi } f(\mathbf{d}, {\mathbf{\beta}})\right]$$,
which is the numerator of equation [2] (part of the likelihood, the denominator of likelihood is constant, and we ignore it), times the prior density $\pi(\beta)$.

The function takes a large number of parameters, some of which include:

- `gamma_val` : Gamma values.
- `gamma_vals_mean` : Mean of gamma values.
- `theta_vals` : Theta values.
- `site_precisions` : Precision of site data.
- `alpha` : Alpha values.
- `sol` : Solution values from optimization.
- And several other parameters.

Inside the function, the `gamma_val` is flattened and a few initial computations are made. `X_zero` is calculated using `gamma_val`, `forestArea_2017_ha`, and `norm_fac`. `X_dym` is calculated using `alpha_p_Adym`, `X_zero`, `Bdym`, and `omega`.

The function then modifies the solution of `X` (i.e., `sol.value(X)`) for further computation. This modification includes shifting and scaling operations on the `X` matrix.

The function computes the log-density in three steps:

- `term_1` is calculated as the negative sum of the product of `ds_vect`, `sol.value(Ua)`, and `zeta`, all divided by 2.
- `term_2` is computed as the sum of the product of `ds_vect`, `pf`, and the difference between successive elements of `X_dym`.
- `term_3` is calculated as the sum of the product of `ds_vect` and the sum of `z_shifted_X`.

The objective value `obj_val` is the sum of `term_1`, `term_2`, and `term_3`.

The function also computes `norm_log_prob` as the negative half of the dot product of `gamma_val_dev` and the dot product of `site_precisions` and `gamma_val_dev`.

The log density value `log_density_val` is calculated as `-1.0 / xi * obj_val + norm_log_prob`.

If the global `_DEBUG` flag is set, the function prints out the values of `term_1`, `term_2`, `term_3`, `obj_val`, `norm_log_prob`, and `log_density_val`.

Finally, the function returns the `log_density_val`.


In [3]:
def log_density_function(gamma_val,
                         gamma_vals_mean,
                         theta_vals,
                         site_precisions,
                         alpha,
                         sol,
                         X,
                         Ua,
                         Up,
                         zbar_2017,
                         forestArea_2017_ha,
                         norm_fac,
                         alpha_p_Adym,
                         Bdym,
                         leng,
                         T,
                         ds_vect,
                         zeta,
                         xi,
                         kappa,
                         pa,
                         pf,
                         ):
    """
    Define a function to evaluate log-density of the objective/posterior distribution
    Some of the input parameters are updated at each cycle of the outer loop (optimization loop),
    and it becomes then easier/cheaper to udpate the function stamp and keep it separate here
    """
    N          = X.shape[1] - 1

    gamma_val  = np.asarray(gamma_val).flatten()
    gamma_size = gamma_val.size
    x0_vals    = gamma_val.T.dot(forestArea_2017_ha) / norm_fac
    X_zero     = np.sum(x0_vals) * np.ones(leng)

    # shifted_X = zbar_2017 - sol.value(X)[0:gamma_size, :-1]
    shifted_X  = sol.value(X)[0: gamma_size, :-1].copy()
    for j in range(N):
        shifted_X[:, j]  = zbar_2017 - shifted_X[:, j]
    omega      = np.dot(gamma_val, alpha * shifted_X - sol.value(Up))

    X_dym      = np.zeros(T+1)
    X_dym[0]   = np.sum(x0_vals)
    X_dym[1: ] = alpha_p_Adym * X_zero  + np.dot(Bdym, omega.T)

    z_shifted_X = sol.value(X)[0: gamma_size, :].copy()
    scl = pa * theta_vals - pf * kappa
    for j in range(N+1): z_shifted_X [:, j] *= scl

    term_1 = - casadi.sum2(np.reshape(ds_vect[0: T], (1, T)) * sol.value(Ua) * zeta / 2 )
    term_2 = casadi.sum2(np.reshape(ds_vect[0: T], (1, T)) * pf * (X_dym[1: ] - X_dym[0: -1])) # X_dym is for time t+1
    term_3 = casadi.sum2(np.reshape(ds_vect, (1, N+1)) * casadi.sum1(z_shifted_X)) # z_shifted_x is for time t

    obj_val = term_1 + term_2 + term_3

    gamma_val_dev   = gamma_val - gamma_vals_mean
    norm_log_prob   =   - 0.5 * np.dot(gamma_val_dev,
                                       site_precisions.dot(gamma_val_dev)
                                       )
    log_density_val = -1.0  / xi * obj_val + norm_log_prob

    if _DEBUG:
        print("Term 1: ", term_1)
        print("Term 2: ", term_2)
        print("Term 3: ", term_3)
        print("obj_val: ", obj_val)
        print("norm_log_prob", norm_log_prob)
        print("log_density_val", log_density_val)

    return log_density_val

# 3 Using Hamiltonian Monte Carlo to Approximate Posterior Distribution

We seek to non-parametrically estimate the log of equation [2] excluding the constant term (denominator), using Hamiltonian Monte Carlo
$$
\frac {\exp\left[ - {\frac 1 \xi } f({ d},  { \beta} )\right]}
{\int_{\mathcal B} \exp\left[ - {\frac 1 \xi } f({ d}, \tilde  { \beta})\right]d\pi( \tilde { \beta} )} \pi(\beta)
$$


## 3.1 Overview of Algorithm
The estimation begins by loading data related to the sites and performing some preliminary calculations on gamma values. It then calculates the mean and covariances from the site data. It sets up various data structures and matrices to hold information as it iterates through the main loop.

The main loop is a convergence loop that continues until the maximum number of iterations is reached or the error is below a specified tolerance. In each loop, the function:

- Updates the values of x0 and constructs matrices A and D using the current gamma values.

- Defines the right-hand side of an equation to be solved by a solver.

- Sets up and solves an optimization problem using the CasADi `Opti` class.

- Generates samples using the Hamiltonian Monte Carlo method. It then updates the gamma values and computes an error metric for convergence.

- If the convergence criterion is met, the function exits the loop, otherwise, it continues to the next iteration.

The illustration concludes by sampling the final distribution densely and returning the results.

## 3.2 Configuration and Settings
- `norm_fac`, `delta_t`, `alpha`, `kappa`, `pf`, `pa`, `xi`, `zeta`: These seem to be configurations or settings for the model.

- `max_iter`, `tol`, `T`, `N`: These are parameters that control the iterations in the optimization process.

- `sample_size`, `mode_as_solution`, `final_sample_size`: These parameters relate to the sampling process for the Hamiltonian Monte Carlo method.





In [4]:
# Configurations/Settings
norm_fac          = 1e9
delta_t           = 0.02
alpha             = 0.045007414
kappa             = 2.094215255
pf                = 20.76
pa                = 44.75
xi                = 0.01
zeta              = 1.66e-4*1e9  # zeta := 1.66e-4*norm_fac  #
#
max_iter          = 200
tol               = 0.01    # convergence tolerance
T                 = 200
N                 = 200
#
sample_size       = 1000   # simulations before convergence (to evaluate the mean)
mode_as_solution  = False   # If true, use the mode (point of high probability) as solution for gamma
final_sample_size = 100_00 # number of samples to collect after convergence

# Evaluate Gamma values ()
gamma_1_vals  = gamma -  gammaSD
gamma_2_vals  = gamma +  gammaSD
gamma_size    = gamma.size

# Evaluate mean and covariances from site data
site_stdev       = gammaSD
site_covariances = np.diag(np.power(site_stdev, 2))
site_precisions  = np.linalg.inv(site_covariances)
site_mean        = gamma_1_vals/2 + gamma_2_vals/2

# Retrieve z data for selected site(s)
site_z_vals  = z_2017

# Initialize Gamma Values
gamma_vals      = gamma.copy()
gamma_vals_mean = gamma.copy()
gamma_vals_old  = gamma.copy()

# Theta Values
theta_vals  = theta

# Householder to track sampled gamma values
# gamma_vals_tracker       = np.empty((gamma_vals.size, sample_size+1))
# gamma_vals_tracker[:, 0] = gamma_vals.copy()
gamma_vals_tracker = [gamma_vals.copy()]

# Collected Ensembles over all iterations; dictionary indexed by iteration number
collected_ensembles = {}

# Track error over iterations
error_tracker = []

# Update this parameter (leng) once figured out where it is coming from
leng = 200
arr  = np.cumsum(
            np.triu(
            np.ones((leng, leng))
        ),
        axis=1,
).T
Bdym         = (1-alpha) ** (arr-1)
Bdym[Bdym>1] = 0.0
Adym         = np.arange(1, leng+1)
alpha_p_Adym = np.power(1-alpha, Adym)

# Initialize Blocks of the A matrix those won't change
A  = np.zeros((gamma_size+2, gamma_size+2))
Ax = np.zeros(gamma_size+2)

# Construct Matrix B
B = np.eye(N=gamma_size+2, M=gamma_size, k=0)
B = casadi.sparsify(B)

# Construct Matrxi D constant blocks
D  = np.zeros((gamma_size+2, gamma_size))

# time step!
dt = T / N

# Other placeholders!
ds_vect = np.exp(- delta_t * np.arange(N+1) * dt)
ds_vect = np.reshape(ds_vect, (ds_vect.size, 1))

# Results dictionary
results = dict(gamma_size=gamma_size, tol=tol, T=T, N=N, norm_fac=norm_fac, delta_t=delta_t, alpha=alpha, kappa=kappa,
    pf=pf, pa=pa, xi=xi, zeta=zeta, sample_size=sample_size, final_sample_size=final_sample_size, mode_as_solution=mode_as_solution,)

## 3.3 Hamiltonian Monte Carlo

HMC stands for Hamiltonian Monte Carlo, a type of Markov Chain Monte Carlo (MCMC) method used in computational statistics to draw samples from a probability distribution, particularly high-dimensional ones.

Below is a descirption of the HMC
1. **Initialization**: Start with an initial guess for $\beta$, typically denoted as $\bar{\beta}$. Simultaneously, generate a random value for $\mu$ from its marginal distribution.

2. **Symplectic Integration**: Use a discrete approximation method that preserves volume, also known as a symplectic integrator, to simulate the Hamiltonian dynamics. This process involves choosing a step size and running the simulation for a predetermined number of steps, typically around 20 but can be tailored based on the problem at hand. The output from this step is a new proposed point in the parameter space, denoted as $(\beta^*,\mu^*)$.

3. **Metropolis Acceptance**: The Metropolis algorithm is then employed to decide whether to accept or reject the proposed point. The acceptance probability is given by $\min\{1, \exp\left(H(\beta,\mu)-H(\beta^*,\mu^*)\right)\}$, with discretization allowing for a probability less than 1.
    - It is important to note that the volume-preservation property of the symplectic integrator guarantees that the Metropolis update leaves the canonical distribution for $(\beta, \mu)$ invariant.

4. **Iteration**: Generate another random value for $\mu'$ and repeat the steps 2 and 3 starting from the new point $(\beta^*,\mu')$. If the proposed point is rejected in the Metropolis step, we stick with the current $\beta$ and repeat the process.

5. **Decision Recalculation**: After each full iteration, the decision $d$ is recalculated.

6. **Convergence**: The process from steps 2 to 5 is repeated until the average $\beta$ value converges to a stable point.

7. **Final Distribution**: Lastly, the HMC is run for a large number of iterations (e.g., 10,000) to produce the final distorted distribution. This comprehensive run allows for more robust estimation of the distribution.


In [5]:
# Initialize error & iteration counter
error = np.infty
cntr = 0

# Loop until convergence
while cntr < max_iter and error > tol:

    # Update x0, initial distribution of carbon absorption of the amazon
    x0_vals = gamma_vals * forestArea_2017_ha / norm_fac

    # Construct Matrix A from new gamma_vals
    A[: -2, :]        = 0.0
    Ax[0: gamma_size] = - alpha * gamma_vals[0: gamma_size]
    Ax[-1]            = alpha * np.sum(gamma_vals * zbar_2017)
    Ax[-2]            = - alpha
    A[-2, :]          = Ax
    A[-1, :]          = 0.0
    A = casadi.sparsify(A)

    # Construct Matrix D from new gamma_vals
    D[:, :]  = 0.0
    D[-2, :] = -gamma_vals
    D = casadi.sparsify(D)

    # Define the right hand side (symbolic here) as a function of gamma
    gamma = casadi.MX.sym('gamma' , gamma_size+2) # state
    up    = casadi.MX.sym('up', gamma_size) # control
    um    = casadi.MX.sym('um', gamma_size) # control

    rhs = (A @ gamma + B @ (up-um) + D @ up) * dt + gamma
    f = casadi.Function('f', [gamma, um, up], [rhs]) # dynamics law of motion for state

    ## Define an optimizer and initialize it, and set constraints
    opti = casadi.Opti()

    # Decision variables for states
    X = opti.variable(gamma_size+2, N+1)

    # Aliases for states
    Up = opti.variable(gamma_size, N)
    Um = opti.variable(gamma_size, N)
    Ua = opti.variable(1, N)

    # Parameter for initial state
    ic = opti.parameter(gamma_size+2)

    # Gap-closing shooting constraints
    for k in range(N):
        opti.subject_to(X[:, k+1] == f(X[:, k], Um[:, k], Up[:, k]))

    # Initial and terminal constraints
    opti.subject_to(X[:, 0] == ic)
    opti.subject_to(opti.bounded(0,X[0: gamma_size, :],zbar_2017[0: gamma_size]))

    # Objective: regularization of controls
    for k in range(gamma_size):
        opti.subject_to(opti.bounded(0, Um[k,:], casadi.inf))
        opti.subject_to(opti.bounded(0, Up[k,:], casadi.inf))

    opti.subject_to(Ua == casadi.sum1(Up+Um)**2)

    # Set teh optimization problem
    term1 = casadi.sum2(ds_vect[0: N, :].T * Ua * zeta / 2)
    term2 = - casadi.sum2(ds_vect[0: N, :].T * (pf * (X[-2, 1: ] - X[-2, 0 :-1])))
    term3 = - casadi.sum2(ds_vect.T * casadi.sum1( (pa * theta_vals - pf * kappa ) * X[0: gamma_size, :] ))

    opti.minimize(term1 + term2 + term3)

    # Solve optimization problem
    options               = dict()
    options["print_time"] = False
    options["expand"]     = True
    options["ipopt"]      = {'print_level':                      0,
                                'fast_step_computation':            'yes',
                                'mu_allow_fast_monotone_decrease':  'yes',
                                'warm_start_init_point':            'yes',
                                }
    opti.solver('ipopt', options)

    opti.set_value(ic, casadi.vertcat(site_z_vals, np.sum(x0_vals), 1),)

    if _DEBUG:
        print("ic: ", ic)
        print("site_z_vals: ", site_z_vals)
        print("x0_vals: ", x0_vals)
        print("casadi.vertcat(site_z_vals,np.sum(x0_vals),1): ", casadi.vertcat(site_z_vals,np.sum(x0_vals),1))
    sol = opti.solve()

    if _DEBUG:
        print("sol.value(X)", sol.value(X))
        print("sol.value(Ua)", sol.value(Ua))
        print("sol.value(Up)", sol.value(Up))
        print("sol.value(Um)", sol.value(Um))


    ## Start Sampling
    # Update signature of log density evaluator
    log_density = lambda gamma_val: log_density_function(gamma_val=gamma_val, gamma_vals_mean=gamma_vals_mean, theta_vals=theta_vals,
                   site_precisions=site_precisions, alpha=alpha, sol=sol, X=X, Ua=Ua, Up=Up, zbar_2017=zbar_2017,
                   forestArea_2017_ha=forestArea_2017_ha, norm_fac=norm_fac, alpha_p_Adym=alpha_p_Adym,
                   Bdym=Bdym, leng=leng, T=T, ds_vect=ds_vect, zeta=zeta, xi=xi, kappa=kappa, pa=pa, pf=pf)

    # Create MCMC sampler & sample, then calculate diagnostics
    sampler = create_hmc_sampler(
        size=gamma_size,
        log_density=log_density,
        burn_in=100,
        mix_in=2,
        symplectic_integrator='verlet',
        symplectic_integrator_stepsize=1e-1,
        symplectic_integrator_num_steps=3,
        mass_matrix=1e+1,
        constraint_test=lambda x: True if np.all(x>=0) else False,
    )
    gamma_post_samples = sampler.sample(
        sample_size=sample_size,
        initial_state=gamma_vals,
        verbose=True,
    )
    gamma_post_samples = np.asarray(gamma_post_samples)

    # Update ensemble/tracker
    collected_ensembles.update({cntr: gamma_post_samples.copy()})

    # Update gamma value
    weight     = 0.25  # <-- Not sure how this linear combination weighting helps!
    if mode_as_solution:
        raise NotImplementedError("We will consider this in the future; trace sampled points and keep track of objective values to pick one with highest prob. ")

    else:
        gamma_vals = weight * np.mean(gamma_post_samples, axis=0 ) + (1-weight) * gamma_vals_old
    gamma_vals_tracker.append(gamma_vals.copy())

    # Evaluate error for convergence check
    error = np.max(np.abs(gamma_vals_old-gamma_vals) / gamma_vals_old)
    error_tracker.append(error)
    print(f"Iteration [{cntr+1:4d}]: Error = {error}")

    # Exchange gamma values (for future weighting/update & error evaluation)
    gamma_vals_old = gamma_vals

    # Increase the counter
    cntr += 1

    results.update({'cntr': cntr,
                    'error_tracker':np.asarray(error_tracker),
                    'gamma_vals_tracker': np.asarray(gamma_vals_tracker),
                    'collected_ensembles':collected_ensembles,
                    })
    pickle.dump(results, open('results.pcl', 'wb'))

    # Extensive plotting for monitoring; not needed really!
    if False:
        plt.plot(gamma_vals_tracker[-2], label=r'Old $\gamma$')
        plt.plot(gamma_vals_tracker[-1], label=r'New $\gamma$')
        plt.legend()
        plt.show()

        for j in range(gamma_size):
            plt.hist(gamma_post_samples[:, j], bins=50)
            plt.title(f"Iteration {cntr}; Site {j+1}")
            plt.show()

print("Terminated. Sampling the final distribution")
# Sample (densly) the final distribution
final_sample = sampler.sample(
    sample_size=final_sample_size,
    initial_state=gamma_vals,
    verbose=True,
)
final_sample = np.asarray(final_sample)
results.update({'final_sample': final_sample})
pickle.dump(results, open('results.pcl', 'wb'))




ic:  opti0_p_1
site_z_vals:  [6.39629863e-03 6.67577070e-04 1.60344839e-02 9.49710184e-03
 6.04574833e-03 2.70778687e-03 7.79948366e-03 5.73571943e-05
 6.77236678e-03 1.30112628e-03]
x0_vals:  [20.85421657 24.91904887  8.3397239   4.13008771  1.21226365 46.76481105
  6.74187483 21.82873633 31.06208981 31.20290148]
casadi.vertcat(site_z_vals,np.sum(x0_vals),1):  [0.0063963, 0.000667577, 0.0160345, 0.0094971, 0.00604575, 0.00270779, 0.00779948, 5.73572e-05, 0.00677237, 0.00130113, 197.056, 1]



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

sol.value(X) [[6.39629863e-03 6.39625350e-03 5.74463986e-03 ... 4.01704222e-08
  2.86738538e-08 1.17041808e-08]
 [6.67577070e-04 6.12690580e-08 3.74046994e-08 ... 2.41594358e-08
  1.63326347e-08 4.77883297e-09]
 [1.60344839e-02 1.60344800e-02 1.60344739e-02 ... 6.13198959e-08
  4.48481458e-08 2.08376901e-08]
 ...
 [1.30112628e-03 7.01132658e-08 4.09636820e-08 ... 3.66683640e-08
  2.61144249e-08 1.02133636e-08]
 [1.97055754e+02 1.97055807e+02 1.97238262e+02 ... 2.23389085e+02
  2.23389313e+02 2.23389537e+02]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]]


Term 1:  -14.9893
Term 2:  -0.37983
Term 3:  30.0684
obj_val:  14.6993
norm_log_prob -25.334491304640917
log_density_val -1495.26
Term 1:  -14.9893
Term 2:  -0.238705
Term 3:  30.0684
obj_val:  14.8404
norm_log_prob -25.333639145021916
log_density_val -1509.37
Term 1:  -14.9893
Term 2:  -0.294216
Term 3:  30.0684
obj_val:  14.7849
norm_log_prob -25.34225968277909
log_density_val -1503.83
Term 1:  -14.9893
Term 2:  -0.294217
Term 3:  30.0684
obj_val:  14.7849
norm_log_prob -25.342259776091204
log_density_val -1503.83
Term 1:  -14.9893
Term 2:  -0.294217
Term 3:  30.0684
obj_val:  14.7849
norm_log_prob -25.342259703273697
log_density_val -1503.83
Term 1:  -14.9893
Term 2:  -0.294217
Term 3:  30.0684
obj_val:  14.7849
norm_log_prob -25.342259776091204
log_density_val -1503.83
Term 1:  -14.9893
Term 2:  -0.294215
Term 3:  30.0684
obj_val:  14.7849
norm_log_prob -25.342259662945136
log_density_val -1503.83
Term 1:  -14.9893
Term 2:  -0.294217
Term 3:  30.0684
obj_val:  14.7849
norm_log_prob

Term 1:  -14.9893
Term 2:  334.726
Term 3:  30.0684
obj_val:  349.805
norm_log_prob -3.7674489504787985e-07
log_density_val -34980.5
Term 1:  -14.9893
Term 2:  334.726
Term 3:  30.0684
obj_val:  349.805
norm_log_prob -3.767246721142109e-07
log_density_val -34980.5
Term 1:  -14.9893
Term 2:  334.726
Term 3:  30.0684
obj_val:  349.805
norm_log_prob -3.7674489504787985e-07
log_density_val -34980.5
Term 1:  -14.9893
Term 2:  334.726
Term 3:  30.0684
obj_val:  349.805
norm_log_prob -3.7674184780163807e-07
log_density_val -34980.5
Term 1:  -14.9893
Term 2:  334.726
Term 3:  30.0684
obj_val:  349.805
norm_log_prob -3.7674489504787985e-07
log_density_val -34980.5
Term 1:  -14.9893
Term 2:  334.726
Term 3:  30.0684
obj_val:  349.805
norm_log_prob -3.7675083524980123e-07
log_density_val -34980.5
Term 1:  -14.9893
Term 2:  334.726
Term 3:  30.0684
obj_val:  349.805
norm_log_prob -3.7674489504787985e-07
log_density_val -34980.5
Term 1:  -14.9893
Term 2:  334.726
Term 3:  30.0684
obj_val:  349.805


Term 1:  -14.9893
Term 2:  334.682
Term 3:  30.0684
obj_val:  349.761
norm_log_prob -1.4433598944106693e-06
log_density_val -34976.1
Term 1:  -14.9893
Term 2:  334.682
Term 3:  30.0684
obj_val:  349.761
norm_log_prob -1.4433493527497566e-06
log_density_val -34976.1
Term 1:  -14.9893
Term 2:  334.682
Term 3:  30.0684
obj_val:  349.761
norm_log_prob -1.4433355952607966e-06
log_density_val -34976.1
Term 1:  -14.9893
Term 2:  334.682
Term 3:  30.0684
obj_val:  349.761
norm_log_prob -1.4433493527497566e-06
log_density_val -34976.1
Term 1:  -14.9893
Term 2:  334.67
Term 3:  30.0684
obj_val:  349.749
norm_log_prob -1.6345568198348132e-06
log_density_val -34974.9
Term 1:  -14.9893
Term 2:  334.67
Term 3:  30.0684
obj_val:  349.749
norm_log_prob -1.6345886308842713e-06
log_density_val -34974.9
Term 1:  -14.9893
Term 2:  334.67
Term 3:  30.0684
obj_val:  349.749
norm_log_prob -1.6345922444141096e-06
log_density_val -34974.9
Term 1:  -14.9893
Term 2:  334.67
Term 3:  30.0684
obj_val:  349.749
nor

...

Term 1:  -14.9893
Term 2:  329.326
Term 3:  30.0684
obj_val:  344.405
norm_log_prob -0.004778722844076898
log_density_val -34440.5
Term 1:  -14.9893
Term 2:  329.26
Term 3:  30.0684
obj_val:  344.339
norm_log_prob -0.004914375034081849
log_density_val -34433.9
HMC Iteration [ 145/2100]; Accept Prob: 1.00; --> Accepted? True  Term 1:  -14.9893
Term 2:  329.27
Term 3:  30.0684
obj_val:  344.349
norm_log_prob -0.004892542545274441
log_density_val -34434.9
Term 1:  -14.9893
Term 2:  329.27
Term 3:  30.0684
obj_val:  344.349
norm_log_prob -0.004892543534622775
log_density_val -34434.9
Term 1:  -14.9893
Term 2:  329.27
Term 3:  30.0684
obj_val:  344.349
norm_log_prob -0.0048925435139844245
log_density_val -34434.9
Term 1:  -14.9893
Term 2:  329.27
Term 3:  30.0684
obj_val:  344.349
norm_log_prob -0.004892543534622775
log_density_val -34434.9
Term 1:  -14.9893
Term 2:  329.27
Term 3:  30.0684
obj_val:  344.349
norm_log_prob -0.004892539694318576
log_density_val -34434.9
Term 1:  -14.9893
Term

Term 1:  -14.9893
Term 2:  329.28
Term 3:  30.0684
obj_val:  344.359
norm_log_prob -0.004856136616435849
log_density_val -34435.9
Term 1:  -14.9893
Term 2:  329.28
Term 3:  30.0684
obj_val:  344.359
norm_log_prob -0.004856136582671129
log_density_val -34435.9
Term 1:  -14.9893
Term 2:  329.28
Term 3:  30.0684
obj_val:  344.359
norm_log_prob -0.004856136616435849
log_density_val -34435.9
Term 1:  -14.9893
Term 2:  329.28
Term 3:  30.0684
obj_val:  344.359
norm_log_prob -0.004856132783247049
log_density_val -34435.9
Term 1:  -14.9893
Term 2:  329.28
Term 3:  30.0684
obj_val:  344.359
norm_log_prob -0.004856136616435849
log_density_val -34435.9
Term 1:  -14.9893
Term 2:  329.28
Term 3:  30.0684
obj_val:  344.359
norm_log_prob -0.004856134665368308
log_density_val -34435.9
Term 1:  -14.9893
Term 2:  329.28
Term 3:  30.0684
obj_val:  344.359
norm_log_prob -0.004856136616435849
log_density_val -34435.9
Term 1:  -14.9893
Term 2:  329.28
Term 3:  30.0684
obj_val:  344.359
norm_log_prob -0.0048

Term 1:  -14.9893
Term 2:  329.221
Term 3:  30.0684
obj_val:  344.301
norm_log_prob -0.0049537165722350375
log_density_val -34430.1
Term 1:  -14.9893
Term 2:  329.221
Term 3:  30.0684
obj_val:  344.301
norm_log_prob -0.004953720424586411
log_density_val -34430.1
Term 1:  -14.9893
Term 2:  329.221
Term 3:  30.0684
obj_val:  344.301
norm_log_prob -0.004953718427903441
log_density_val -34430.1
Term 1:  -14.9893
Term 2:  329.221
Term 3:  30.0684
obj_val:  344.301
norm_log_prob -0.004953720424586411
log_density_val -34430.1
Term 1:  -14.9893
Term 2:  329.221
Term 3:  30.0684
obj_val:  344.301
norm_log_prob -0.004953717566849922
log_density_val -34430.1
Term 1:  -14.9893
Term 2:  329.221
Term 3:  30.0684
obj_val:  344.301
norm_log_prob -0.004953720424586411
log_density_val -34430.1
Term 1:  -14.9893
Term 2:  329.221
Term 3:  30.0684
obj_val:  344.301
norm_log_prob -0.004953719922943719
log_density_val -34430.1
Term 1:  -14.9893
Term 2:  329.221
Term 3:  30.0684
obj_val:  344.301
norm_log_pro

Term 1:  -14.9893
Term 2:  329.183
Term 3:  30.0684
obj_val:  344.262
norm_log_prob -0.0049836713463863825
log_density_val -34426.2
Term 1:  -14.9893
Term 2:  329.183
Term 3:  30.0684
obj_val:  344.262
norm_log_prob -0.004983668583918329
log_density_val -34426.2
Term 1:  -14.9893
Term 2:  329.183
Term 3:  30.0684
obj_val:  344.262
norm_log_prob -0.0049836713463863825
log_density_val -34426.2
Term 1:  -14.9893
Term 2:  329.183
Term 3:  30.0684
obj_val:  344.262
norm_log_prob -0.004983670838688852
log_density_val -34426.2
Term 1:  -14.9893
Term 2:  329.183
Term 3:  30.0684
obj_val:  344.262
norm_log_prob -0.0049836713463863825
log_density_val -34426.2
Term 1:  -14.9893
Term 2:  329.183
Term 3:  30.0684
obj_val:  344.262
norm_log_prob -0.004983670198326282
log_density_val -34426.2
Term 1:  -14.9893
Term 2:  329.183
Term 3:  30.0684
obj_val:  344.262
norm_log_prob -0.0049836713463863825
log_density_val -34426.2
Term 1:  -14.9893
Term 2:  329.183
Term 3:  30.0684
obj_val:  344.262
norm_log_

SystemError: <built-in function OptiSol_value> returned a result with an error set

## 3.4 Plot Results

In [ ]:
# Plot Error Results
plt.plot(results['error_tracker'])
plt.xlabel("Iteration")
plt.ylabel("Error")
plt.show()

In [ ]:
# Plot Gamma Estimate Update
for j in range(results['gamma_size']):
    plt.plot(results['gamma_vals_tracker'][:, j], label=r"$\gamma_{%d}$"%(j+1))
plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0)
plt.show()

In [ ]:
# Plot Gamma Estimate Update
for j in range(results['gamma_size']):
    plt.plot(results['gamma_vals_tracker'][:, j], label=r"$\gamma_{%d}$"%(j+1))
plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0)
plt.show()

In [ ]:
# Plot Histograms
for itr in results['collected_ensembles'].keys():
    for j in range(results['gamma_size']):
        plt.hist(results['collected_ensembles'][itr][:, j], bins=100)
        plt.title(f"Iteration {itr+1}; Site {j+1}")
        plt.show()

In [ ]:
# Plot Histogram of the final sample
for j in range(results['gamma_size']):
    plt.hist(results['final_sample'][:, j], bins=100)
    plt.title(f"Final Sample; Site {j+1}")
    plt.show()